In [2]:
!unzip dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/jack of diamonds/157.jpg  
  inflating: dataset/jack of diamonds/158.jpg  
  inflating: dataset/jack of diamonds/159.jpg  
  inflating: dataset/jack of diamonds/160.jpg  
   creating: dataset/jack of hearts/
  inflating: dataset/jack of hearts/001.jpg  
  inflating: dataset/jack of hearts/002.jpg  
  inflating: dataset/jack of hearts/003.jpg  
  inflating: dataset/jack of hearts/004.jpg  
  inflating: dataset/jack of hearts/005.jpg  
  inflating: dataset/jack of hearts/006.jpg  
  inflating: dataset/jack of hearts/007.jpg  
  inflating: dataset/jack of hearts/008.jpg  
  inflating: dataset/jack of hearts/009.jpg  
  inflating: dataset/jack of hearts/010.jpg  
  inflating: dataset/jack of hearts/011.jpg  
  inflating: dataset/jack of hearts/012.jpg  
  inflating: dataset/jack of hearts/013.jpg  
  inflating: dataset/jack of hearts/014.jpg  
  inflating: dataset/jack of hearts/015.jpg  
  inflating: dataset/jack of h

In [ ]:
!pip install sympy==1.11.1  #only and only if the imports dont work even after restarting runtime, otherwise skip this.

Training phase

In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import torch.nn as nn
import torch.optim as optim
from PIL import Image
d = 0

In [5]:
img_transformer = transforms.Compose([
    transforms.Resize((128, 128)),   # Resize to uniform size
    transforms.ToTensor(),           # Convert to tensor
])

In [6]:
dataset = datasets.ImageFolder(root="dataset", transform=img_transformer)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
print("\nClasses:", dataset.classes)


Classes: ['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two of hearts', 'two of spades']


In [7]:
save_model_name = input("Save the model as: _______.pth: ")

l_r = 0.001
try:
    l_r = float(input("\n\033[34mLearning rate [default lr=0.001]\033[0m \nthis value controls how big a step the optimizer takes when updating model weights during training.\nLess the lr value more the training time, more the accuracy\nEnter the lr value or simply press enter to proceed with default value: "))
except:
    l_r = 0.001
try:
   epc = int(input("\n\033[34mEpochs [default: 10]\033[0m \n(An epoch is one complete pass through the entire training dataset by the model)\nEnter the number of epochs to execute or simply press enter to proceed with default value: "))
except:
   epc = 10


Save the model as: _______.pth: modelx

Learning rate [default lr=0.001] 
this value controls how big a step the optimizer takes when updating model weights during training.
Less the lr value more the training time, more the accuracy
Enter the lr value or simply press enter to proceed with default value: 

Epochs [default: 10] 
(An epoch is one complete pass through the entire training dataset by the model)
Enter the number of epochs to execute or simply press enter to proceed with default value: 5


In [8]:
class CardCNN(nn.Module):
    def __init__(self, num_classes,  colour_channels=3):
        super(CardCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d( colour_channels, 32, kernel_size=3, padding=1),  # Input 3x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 64x32x32
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device} \n")
model = CardCNN(num_classes=len(dataset.classes), colour_channels=3).to(device)      #default 3 (RGB), change to 1 for grayscale

criterion = nn.CrossEntropyLoss()
if l_r == "":
    optimizer = optim.Adam(model.parameters(), lr=0.001)
else:
    optimizer = optim.Adam(model.parameters(), lr=l_r)


Using device: cuda 



In [11]:
d =0
start_time = time.time()
print(f"\033[32mtraining started...\033[0m \ndetails:\n lr count: {l_r}\n epoch count: {epc}\n")

try:
    for epoch in range(epc):
        print(f"\033[33mEpoch: {epoch+1}\033[0m")
        t = time.time()
        total_loss = 0
        model.train()
        for images, labels in dataloader:

            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            print(f"  loss: {loss.item()}")
            total_loss += loss.item()

        print(f"Epoch - {epoch+1}:\n Total loss: {total_loss:.4f}")
        t = (time.time() - t)
        print(f" time taken for epoch {epoch+1}: {(t)} seconds")
        d +=t

    print(f"\n\033[32mTraining completed in {time.time() - start_time:.2f} seconds \nAverage time per epoch: {(d)/epc}\033[0m")

    torch.save(model.state_dict(), f"{save_model_name}.pth")
    print(f"model saved as: {save_model_name}.pth")
except Exception as e:
    print(f"error: {e}")

training started... 
details:
 lr count: 0.001
 epoch count: 5

Epoch: 1
  loss: 0.20596469938755035
  loss: 0.07107431441545486
  loss: 0.11567723751068115
  loss: 0.3460467457771301
  loss: 0.08019975572824478
  loss: 0.1066148653626442
  loss: 0.03291003406047821
  loss: 0.15797880291938782
  loss: 0.1499786227941513
  loss: 0.06499990075826645
  loss: 0.5113247036933899
  loss: 0.1427125483751297
  loss: 0.05573917552828789
  loss: 0.09061522781848907
  loss: 0.08591533452272415
  loss: 0.21648244559764862
  loss: 0.35119232535362244
  loss: 0.38813331723213196
  loss: 0.06298134475946426
  loss: 0.09958939999341965
  loss: 0.04913249984383583
  loss: 0.36299628019332886
  loss: 0.17534878849983215
  loss: 0.47465211153030396
  loss: 0.09768659621477127
  loss: 0.11362999677658081
  loss: 0.10504554957151413
  loss: 0.14762566983699799
  loss: 0.013838697224855423
  loss: 0.13827022910118103
  loss: 0.08859999477863312
  loss: 0.12441418319940567
  loss: 0.055290985852479935
  loss

Testing/prediction phase

In [12]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import datasets, transforms

In [13]:
img_transformer = transforms.Compose([
    transforms.Resize((128, 128)),   # Resize to uniform size
    transforms.ToTensor(),           # Convert to tensor
])

In [14]:
#classes = ['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two of hearts', 'two of spades']
dataset = datasets.ImageFolder(root="dataset", transform=img_transformer)
# you get the point...
#print("Classes:", classes)
print("Classes:", dataset.classes)

Classes: ['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two of hearts', 'two of spades']


In [15]:
class CardCNN(nn.Module):                                   # Make sure this matches the original training model class
    def __init__(self, num_classes):
        super(CardCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),     # Input 3x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                                # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                                # 64x32x32
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\033[33mUsing device: {device}\033[0m\n")

model = CardCNN(num_classes=len(dataset.classes))          # Make sure this matches the original training model
try:
    model.load_state_dict(torch.load("modelx.pth", map_location=torch.device('cpu')))      #modify this to your model name
except:
    model.load_state_dict(torch.load(f"{save_model_name}.pth", map_location=torch.device('cpu')))

Using device: cuda



<All keys matched successfully>

In [20]:
model.eval()  # Set model to evaluation mode
image = Image.open("test.png").convert("RGB")               # Replace with your image
image = img_transformer(image).unsqueeze(0)
with torch.no_grad():
    output = model(image)
    predicted_index = output.argmax(1).item()

print(f"\033[32mPredicted class: {dataset.classes[predicted_index]}\033[0m")

image = Image.open("test1.png").convert("RGB")               # Replace with your image
image = img_transformer(image).unsqueeze(0)
with torch.no_grad():
    output = model(image)
    predicted_index = output.argmax(1).item()
print(f"\033[32mPredicted class: {dataset.classes[predicted_index]}\033[0m")

Predicted class:ace of hearts
Predicted class:ace of hearts
